In [18]:
import numpy as np
import matplotlib.pyplot as plt
import skimage.metrics as metrics
import mat73
import pandas as pd
from scipy.stats import entropy

In [19]:
def normalized_mutual_information(image0, image1, *, bins=100):
    r"""Compute the normalized mutual information (NMI).

    The normalized mutual information of :math:`A` and :math:`B` is given by::

    .. math::

        Y(A, B) = \frac{H(A) + H(B)}{H(A, B)}

    where :math:`H(X) := - \sum_{x \in X}{x \log x}` is the entropy.

    It was proposed to be useful in registering images by Colin Studholme and
    colleagues [1]_. It ranges from 1 (perfectly uncorrelated image values)
    to 2 (perfectly correlated image values, whether positively or negatively).

    Parameters
    ----------
    image0, image1 : ndarray
        Images to be compared. The two input images must have the same number
        of dimensions.
    bins : int or sequence of int, optional
        The number of bins along each axis of the joint histogram.

    Returns
    -------
    nmi : float
        The normalized mutual information between the two arrays, computed at
        the granularity given by ``bins``. Higher NMI implies more similar
        input images.

    Raises
    ------
    ValueError
        If the images don't have the same number of dimensions.

    Notes
    -----
    If the two input images are not the same shape, the smaller image is padded
    with zeros.

    References
    ----------
    .. [1] C. Studholme, D.L.G. Hill, & D.J. Hawkes (1999). An overlap
           invariant entropy measure of 3D medical image alignment.
           Pattern Recognition 32(1):71-86
           :DOI:`10.1016/S0031-3203(98)00091-0`
    """
    if image0.ndim != image1.ndim:
        raise ValueError(
            f'NMI requires images of same number of dimensions. '
            f'Got {image0.ndim}D for `image0` and '
            f'{image1.ndim}D for `image1`.'
        )
    if image0.shape != image1.shape:
        max_shape = np.maximum(image0.shape, image1.shape)
        padded0 = _pad_to(image0, max_shape)
        padded1 = _pad_to(image1, max_shape)
    else:
        padded0, padded1 = image0, image1

    hist, bin_edges = np.histogramdd(
        [np.reshape(padded0, -1), np.reshape(padded1, -1)],
        bins=bins,
        density=True,
    )

    H0 = entropy(np.sum(hist, axis=0))
    H1 = entropy(np.sum(hist, axis=1))
    H01 = entropy(np.reshape(hist, -1))

    return (H0 + H1) / H01

In [20]:
file = mat73.loadmat(r'C:\Joe Evans\University\Computing\Summer Project\DeepLearningForJoe\SimulateData\Experiments\regularisation_picker\images3_gaussian3_2500.mat')
clean = file['clean_img']
noisy = file['noisy_img']

In [34]:
ssim = []
mse = []
nmi = []
for i in range(noisy.shape[3]):
    ssim.append(metrics.structural_similarity(clean, noisy[:,:,:,i]))
    mse.append(metrics.mean_squared_error(clean, noisy[:,:,:,i]))
    nmi.append(normalized_mutual_information(clean, noisy[:,:,:,i]))
print(nmi)


d = {'SSIM': ssim, 'MSE': mse, 'NMI': nmi}
df2 = pd.DataFrame(d)
df2

[1.0311876170083383, 1.0353933495373626, 1.0361421819370855, 1.0320488397641823]


,SSIM,MSE,NMI
0,0.181614,0.895590,1.031188
1,0.256452,0.816234,1.035393
2,0.287507,0.826285,1.036142
3,0.244424,0.914574,1.032049
